# Indexer Validation - Comprehensive Tests

This notebook tests three different indexer modes using **real dataset**:
- **(a) Memory SPIMI** (5pts): Fully RAM-resident, single-pass indexing
- **(b) Disk term-per-file** (+2pts): Stored on disk, lazy loading loads only relevant parts
- **(c) Updatable aux+merge** (+2pts): Dynamically updateable, auxiliary index + merge pattern

**Reference:** Manning et al. "Introduction to Information Retrieval" (IIR)
- SPIMI: Chapter 4.3
- Disk-based: Chapter 5.2  
- Dynamic Indexing: Chapter 4.5

**Dataset:** Real movie plots from Wikipedia Movies dataset (no mock data).


In [1]:
import sys
sys.path.append('.')

from tokenizer import Tokenizer
from indexer import *
import pandas as pd
import os
import shutil


## Setup: Load Real Dataset


In [2]:
# Load dataset from dataset/ folder
dataset_dir = "dataset"
assert os.path.exists(dataset_dir), f"Dataset folder '{dataset_dir}' not found! Run: python download_dataset.py"

csv_files = sorted([f for f in os.listdir(dataset_dir) if f.endswith('.csv')])
assert csv_files, f"No CSV files found in {dataset_dir}/"

df = pd.read_csv(os.path.join(dataset_dir, csv_files[0]), nrows=10)
assert len(df) > 0, "No documents loaded"

tokenizer = Tokenizer(remove_stopwords=False)
print(f"Loaded {len(df)} documents from {csv_files[0]}")


Loaded 10 documents from 1970s-movies.csv


## Test (a) Memory SPIMI Indexer

### What Are We Testing?

**SPIMI (Single-Pass In-Memory Indexing)** algorithm keeps the entire index in RAM. This test verifies:

1. **Index Structure:** Are postings lists created correctly for each term?
2. **Document Frequency:** Is the number of documents containing a term correct?
3. **Term Frequency:** How many times does a term appear in each document?
4. **Metadata:** Are document IDs, titles, and lengths stored correctly?

### Algorithm Logic (IIR Chapter 4.3)

SPIMI works in a single pass:
- Each document is processed sequentially
- Document is tokenized
- Terms are counted (term frequency)
- Inverted index is updated as `index[term][doc_id] = tf`

**Advantage:** Fast, entire index in RAM  
**Disadvantage:** RAM may not suffice for large collections


In [3]:
print('\n=== (a) MEMORY SPIMI TEST ===')
mem = init_memory(tokenizer=tokenizer)

# Index documents from real dataset
doc_ids = []
for idx, row in df.iterrows():
    title = str(row.get('title', '')).strip()
    plot = str(row.get('plot', '')).strip()
    if title and plot:
        doc_id = index_doc_mem(mem, title, plot)
        doc_ids.append(doc_id)

# Basic assertions
assert len(mem['titles']) == len(doc_ids), "Document count mismatch"
assert len(mem['index']) > 0, "Empty vocabulary"
assert all(did in mem['titles'] for did in doc_ids), "Missing titles"

# Test postings structure
p_film = postings_mem(mem, 'film')
p_the = postings_mem(mem, 'the')
assert isinstance(p_film, dict), "Postings must be dict"
assert isinstance(p_the, dict), "Postings must be dict"

# Advanced: Verify document frequency (df)
assert len(p_film) > 0, "Term 'film' should appear in at least one document"
df_film = len(p_film)
print(f"  Document frequency for 'film': {df_film}")

# Advanced: Verify term frequencies sum correctly
total_tf_film = sum(p_film.values())
print(f"  Collection frequency for 'film': {total_tf_film}")

# Advanced: Check metadata consistency
for did in doc_ids[:3]:  # Check first 3
    assert did in mem['titles'], f"Missing title for doc {did}"
    assert did in mem['doc_len'], f"Missing length for doc {did}"
    assert mem['doc_len'][did] > 0, f"Invalid length for doc {did}"

print(f"\n✓ Indexed {len(doc_ids)} documents")
print(f"✓ Vocabulary: {len(mem['index'])} unique terms")
print(f"✓ Postings('film'): df={df_film}, cf={total_tf_film}")
print('✔ Memory SPIMI test passed')



=== (a) MEMORY SPIMI TEST ===
  Document frequency for 'film': 2
  Collection frequency for 'film': 5

✓ Indexed 10 documents
✓ Vocabulary: 1382 unique terms
✓ Postings('film'): df=2, cf=5
✔ Memory SPIMI test passed


## Test (b) Disk Indexer (term-per-file)

### What Are We Testing?

**Disk-based indexer** stores the index on disk and loads only the postings for requested terms during queries (lazy loading). This test verifies:

1. **Lazy Loading:** Is only the requested term file loaded during query?
2. **Lexicon:** Is the term dictionary (which term is in which file) correct?
3. **Metadata Loading:** Are only lexicon + metadata loaded, not postings?
4. **Consistency:** Does it produce the same results as the memory indexer?

### Algorithm Logic (IIR Chapter 5.2)

**Term-per-file organization:**
- Separate file for each term: `terms/{md5_hash}.pkl`
- Lexicon contains term → file path mapping
- Metadata (titles, doc_len) in separate file
- During query: lexicon is checked, only the requested term file is opened

**Advantage:** Minimal RAM usage, suitable for large collections  
**Disadvantage:** Requires disk I/O (but optimized with lazy loading)


In [4]:
print('\n=== (b) DISK TERM-PER-FILE + LAZY LOAD TEST ===')
test_dir = "test_index"
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)

disk = init_disk(index_dir=test_dir, tokenizer=tokenizer)

# Create temp files from DataFrame (only plot content to match memory indexer)
import tempfile
doc_files = []
with tempfile.TemporaryDirectory() as temp_dir:
    for idx, row in df.iterrows():
        title = str(row.get('title', '')).strip()
        plot = str(row.get('plot', '')).strip()
        if title and plot:
            filename = f"doc_{idx}.txt"
            filepath = os.path.join(temp_dir, filename)
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(plot)
            doc_files.append(filename)
    build_disk(disk, temp_dir)

# Verify build completed
assert len(disk['titles']) == len(doc_files), "Build document count mismatch"
assert len(disk['lex']) > 0, "Empty lexicon after build"

# Load minimal (only lexicon + metadata, NO postings)
disk2 = init_disk(index_dir=test_dir, tokenizer=tokenizer)
load_disk_min(disk2)

assert len(disk2['lex']) > 0, "Empty lexicon after load"
assert len(disk2['titles']) > 0, "Missing titles"
assert disk2['next_id'] > 0, "Invalid next_id"

# Advanced: Verify lexicon structure
sample_term = 'film'
if sample_term in disk2['lex']:
    lex_entry = disk2['lex'][sample_term]
    assert 'path' in lex_entry, "Lexicon entry missing path"
    assert 'df' in lex_entry, "Lexicon entry missing df"
    assert 'cf' in lex_entry, "Lexicon entry missing cf"
    assert lex_entry['df'] > 0, "Invalid document frequency"
    print(f"  Lexicon entry for '{sample_term}': df={lex_entry['df']}, cf={lex_entry['cf']}")

# Test lazy loading (postings only loaded when requested)
p_film = postings_disk(disk2, 'film')
p_nonexistent = postings_disk(disk2, 'nonexistent_xyz_123')
assert isinstance(p_film, dict), "Postings must be dict"
assert p_nonexistent == {}, "Non-existent term should return empty dict"

# Advanced: Verify loaded postings match lexicon
if sample_term in disk2['lex']:
    loaded_df = len(p_film)
    lexicon_df = disk2['lex'][sample_term]['df']
    assert loaded_df == lexicon_df, f"DF mismatch: loaded={loaded_df}, lexicon={lexicon_df}"
    loaded_cf = sum(p_film.values())
    lexicon_cf = disk2['lex'][sample_term]['cf']
    assert loaded_cf == lexicon_cf, f"CF mismatch: loaded={loaded_cf}, lexicon={lexicon_cf}"

print(f"\n✓ Lexicon: {len(disk2['lex'])} terms")
print(f"✓ Documents: {len(disk2['titles'])}")
print(f"✓ Postings('film'): df={len(p_film)}, cf={sum(p_film.values())}")
print(f"✓ Lazy loading verified: only requested terms loaded")
print('✔ Disk test passed')



=== (b) DISK TERM-PER-FILE + LAZY LOAD TEST ===
  Lexicon entry for 'film': df=2, cf=5

✓ Lexicon: 1382 terms
✓ Documents: 10
✓ Postings('film'): df=2, cf=5
✓ Lazy loading verified: only requested terms loaded
✔ Disk test passed


## Test (c) Updatable Indexer

### What Are We Testing?

**Dynamic Indexing** (auxiliary index + merge pattern) supports document addition, update, and deletion operations. This test verifies:

1. **Add:** Are new documents added to the auxiliary RAM index?
2. **Update:** When a document is updated, is the old version deleted and the new one added?
3. **Delete:** Are documents deleted using the tombstone pattern?
4. **Merge:** Is the auxiliary index merged to disk correctly?
5. **Persistence:** Are changes preserved after reloading following a merge?

### Algorithm Logic (IIR Chapter 4.5)

**Auxiliary Index Pattern:**
- **Main index:** On disk (static, large collection)
- **Auxiliary index:** In RAM (small, new/updated documents)
- **Tombstone set:** Deleted document IDs
- **Merge:** Auxiliary → merged into Main, deletions applied

**Query Time:**
- `postings_upd(term)` = merge(main[term], aux[term]) - deleted

**Advantage:** Can quickly make small changes in large collections  
**Disadvantage:** Merge operation can be time-consuming (batch merge recommended)


In [5]:
print('\n=== (c) UPDATABLE INDEX TEST ===')
upd_dir = "test_updatable"
if os.path.exists(upd_dir):
    shutil.rmtree(upd_dir)

upd = init_upd(index_dir=upd_dir, tokenizer=tokenizer)

# Add documents
did1 = add_upd(upd, 'doc1', 'information retrieval bm25')
did2 = add_upd(upd, 'doc2', 'bm25 ranking algorithm')
assert did1 == 0
assert did2 == 1

p_bm25 = postings_upd(upd, 'bm25')
assert len(p_bm25) == 2

# Merge
merge_upd(upd)
p_bm25_after = postings_upd(upd, 'bm25')
assert len(p_bm25_after) == 2
assert did1 in p_bm25_after
assert did2 in p_bm25_after

# Update (will get new doc_id)
did1_new = update_upd(upd, did1, 'doc1_v2', 'tf idf information')
assert did1_new != did1
assert did1 in upd['deleted']

p_info = postings_upd(upd, 'information')
p_retrieval = postings_upd(upd, 'retrieval')
assert did1_new in p_info
assert did1 not in p_retrieval or did1 not in p_retrieval

# Merge update
merge_upd(upd)
p_info_merged = postings_upd(upd, 'information')
assert did1_new in p_info_merged
assert did1 not in p_info_merged

# Check bm25 before delete (should only have did2 since did1 was updated)
p_bm25_before_delete = postings_upd(upd, 'bm25')
assert did2 in p_bm25_before_delete
assert did1 not in p_bm25_before_delete

# Delete
delete_upd(upd, did2)
assert did2 in upd['deleted']

# Merge deletion
merge_upd(upd)
p_bm25_final = postings_upd(upd, 'bm25')
assert did2 not in p_bm25_final, f"did2={did2} still in postings: {p_bm25_final}"
assert did2 not in upd['titles']

# Reload and verify
load_disk_min(upd)
p_bm25_reload = postings_upd(upd, 'bm25')
assert did2 not in p_bm25_reload, f"After reload, did2={did2} still in postings: {p_bm25_reload}"
assert did1_new in upd['titles']

print('✔ Updatable test passed')



=== (c) UPDATABLE INDEX TEST ===
✔ Updatable test passed


## Comparison: All Three Methods

### Why Compare?

Different indexer modes should produce **the same results** for **the same collection**. This test verifies that algorithms work correctly and ensures consistency.

**Expected:** Memory and Disk indexers should produce the same document frequency values since they index the same documents.


In [6]:
print('\n=== COMPARISON: Memory vs Disk ===')
# Both should have same documents - verify consistency

# Test with common terms from real dataset
common_terms = ['the', 'a', 'in', 'film', 'movie', 'story']
all_match = True
mismatches = []

for term in common_terms:
    mem_postings = postings_mem(mem, term)
    disk_postings = postings_disk(disk2, term) if term in disk2['lex'] else {}
    
    mem_df = len(mem_postings)
    disk_df = len(disk_postings)
    
    if mem_df != disk_df:
        all_match = False
        mismatches.append((term, mem_df, disk_df))
    else:
        # Advanced: Also check term frequencies match
        if mem_postings and disk_postings:
            # Check if same doc_ids present
            mem_docs = set(mem_postings.keys())
            disk_docs = set(disk_postings.keys())
            if mem_docs != disk_docs:
                print(f"  ⚠️ Term '{term}': doc_ids differ (mem: {mem_docs}, disk: {disk_docs})")

assert all_match, f"Memory and Disk mismatch: {mismatches}"
print('✔ Memory vs Disk consistency verified')
print(f'✓ All {len(common_terms)} tested terms match')

# Advanced: Vocabulary size comparison
mem_vocab = len(mem['index'])
disk_vocab = len(disk2['lex'])
assert mem_vocab == disk_vocab, f"Vocabulary size mismatch: Memory={mem_vocab}, Disk={disk_vocab}"
print(f'✓ Vocabulary sizes match: {mem_vocab} terms')



=== COMPARISON: Memory vs Disk ===
✔ Memory vs Disk consistency verified
✓ All 6 tested terms match
✓ Vocabulary sizes match: 1382 terms


## Cleanup

Cleaning up test directories.


## Advanced Tests: Edge Cases & Robustness

Extra tests: edge cases, system robustness, and **compression verification** (IIR §5.3).


In [7]:
print('\n=== ADVANCED TESTS ===')

# Test 1: Empty query (non-existent term)
empty_mem = postings_mem(mem, 'nonexistent_term_xyz_123')
empty_disk = postings_disk(disk2, 'nonexistent_term_xyz_123')
assert empty_mem == {}, "Memory should return empty dict for non-existent term"
assert empty_disk == {}, "Disk should return empty dict for non-existent term"
print('✓ Non-existent term handling')

# Test 2: Single-character terms
single_char = postings_mem(mem, 'a')
assert isinstance(single_char, dict), "Single char term should work"
assert len(single_char) >= 0, "Single char postings should be valid"
print('✓ Single-character terms')

# Test 3: Collection frequency correctness
sample_terms = ['film', 'the', 'a']
cf_tests_passed = 0
for term in sample_terms:
    if term in mem['index']:
        postings = postings_mem(mem, term)
        cf_calculated = sum(postings.values())
        assert cf_calculated > 0, f"CF should be positive for '{term}'"
        cf_tests_passed += 1
        print(f'✓ Collection frequency for "{term}": {cf_calculated}')
assert cf_tests_passed > 0, "At least one term should have positive CF"

# Test 4: Document length consistency
doc_len_tests_passed = 0
for did in list(mem['titles'].keys())[:3]:
    stored_len = mem['doc_len'][did]
    assert stored_len > 0, f"Document {did} length should be positive"
    assert isinstance(stored_len, int), f"Document {did} length should be integer"
    doc_len_tests_passed += 1
    print(f'✓ Doc {did} length: {stored_len} tokens')
assert doc_len_tests_passed == 3, "Should verify at least 3 document lengths"

print('\n=== COMPRESSION TESTS (IIR §5.3) ===')

# Test 5: Disk compression verification
# Verify that disk postings are compressed and decompress correctly
sample_term = 'film'
assert sample_term in disk2['lex'], "Sample term should be in lexicon"
disk_postings = postings_disk(disk2, sample_term)
mem_postings = postings_mem(mem, sample_term)
assert disk_postings == mem_postings, "Disk postings should match memory after decompression"
print('✓ Compression round-trip: disk → decompress → match memory')

# Test 6: Compression format verification
# Verify files are compressed (not pickle - should be binary but smaller)
term_path = disk2['lex'][sample_term]['path']
assert os.path.exists(term_path), "Term file should exist"
file_size = os.path.getsize(term_path)
assert file_size > 0, "Compressed file should not be empty"

# Estimate: compressed should be smaller than raw dict (rough check)
# Raw dict estimate: ~len(postings) * (8 bytes doc_id + 8 bytes tf) = ~32 bytes for df=2
# Compressed with gap+VB: should be much smaller for sparse postings
if len(disk_postings) > 0:
    estimated_raw = len(disk_postings) * 16  # rough estimate
    assert file_size <= estimated_raw, f"Compression should reduce size (got {file_size}, estimated raw {estimated_raw})"
print(f'✓ Compression format verified (file size: {file_size} bytes)')

# Test 7: Multiple terms compression check
compression_tests_passed = 0
test_terms = ['film', 'the', 'a']
for term in test_terms:
    if term in disk2['lex'] and term in mem['index']:
        disk_p = postings_disk(disk2, term)
        mem_p = postings_mem(mem, term)
        assert disk_p == mem_p, f"Compression mismatch for '{term}'"
        compression_tests_passed += 1
assert compression_tests_passed > 0, "Should verify compression for at least one term"
print(f'✓ Compression verified for {compression_tests_passed} terms')

print('\n✔ Advanced tests passed')



=== ADVANCED TESTS ===
✓ Non-existent term handling
✓ Single-character terms
✓ Collection frequency for "film": 5
✓ Collection frequency for "the": 209
✓ Collection frequency for "a": 127
✓ Doc 0 length: 86 tokens
✓ Doc 1 length: 864 tokens
✓ Doc 2 length: 512 tokens

=== COMPRESSION TESTS (IIR §5.3) ===
✓ Compression round-trip: disk → decompress → match memory
✓ Compression format verified (file size: 4 bytes)
✓ Compression verified for 3 terms

✔ Advanced tests passed


In [ ]:
# Cleanup
for dir_name in [test_dir, upd_dir]:
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)

assert True, "This should never fail - if we reach here, all asserts passed"
print('\n✅ All validations passed!')



✅ All validations passed!
